In [6]:

#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime

In [3]:
df=pd.read_csv(r"C:\Users\Equipo\Downloads\credits.csv")


In [4]:
df2=pd.read_csv(r"C:\Users\Equipo\Downloads\movies_dataset (1).csv")

C:\Users\Equipo\AppData\Local\Temp\ipykernel_4796\705048831.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv(r"C:\Users\Equipo\Downloads\movies_dataset (1).csv")


In [2]:
#Se crea la funcion para desanidar.
class desanidar:
    @staticmethod
    def convertir_a_str(valor):
        if isinstance(valor, (list, dict)):
            return json.dumps(valor)
        return str(valor)
    @staticmethod
    def extraer_nombres(valor):
        pattern = r"'name': '([^']*)'"
        coincidencias = re.findall(pattern, valor)
        if len(coincidencias) > 0:
            nombre = coincidencias[0]
            return nombre
        else:
            return None

In [7]:
#Segundo paso para desanidar (Son varios)
df2['belongs_to_collection'] = df2['belongs_to_collection'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [8]:
df2['genres'] = df2['genres'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [11]:
df2['production_companies'] = df2['production_companies'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [10]:
df2['production_countries'] = df2['production_countries'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [9]:
df2['spoken_languages'] = df2['spoken_languages'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [13]:
df['cast'] = df['cast'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [14]:
df['crew'] = df['crew'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [15]:
df.head(5)

,cast,crew,id
0,Tom Hanks,John Lasseter,862
1,Robin Williams,Larry J. Franco,8844
2,Walter Matthau,Howard Deutch,15602
3,Whitney Houston,Forest Whitaker,31357
4,Steve Martin,Alan Silvestri,11862


In [18]:
df2.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,Animation,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,False,7.7,5415.0
1,False,None,65000000,Adventure,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,English,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,Grumpy Old Men Collection,0,Romance,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [19]:
#Se borran las columnas que no sirven a futuro
df2 = df2.dropna(subset=['release_date'])
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df2 = df2.drop(columns=columnas_a_eliminar)

In [23]:
# Convertir las columnas 'budget' y 'revenue' a tipo numérico
df2['budget'] = pd.to_numeric(df2['budget'], errors='coerce')
df2['revenue'] = pd.to_numeric(df2['revenue'], errors='coerce')

# Calcular el retorno de inversión y asignar el valor 0 cuando no haya datos disponibles
df2['return'] = df2['revenue'].div(df2['budget'], fill_value=0)

In [24]:
#Cambio a numerico el tipo de datos de la columna ID
df2['id'] = pd.to_numeric(df2['id'], errors='coerce')

In [25]:
#Observo cuales son las columnas sin ID
df2[df2['id'].isnull()]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return
19730,None,NaN,Carousel Productions,NaN,104.0,Released,NaN,None,None,1,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
29503,None,NaN,Aniplex,NaN,68.0,Released,NaN,None,None,12,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN
35587,None,NaN,Odyssey Media,NaN,82.0,Released,Beware Of Frost Bites,None,None,22,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#Dado que solo son 3, decido eliminarlas
df2 = df2.dropna(subset=['id'])

In [27]:
#Cambio el tipo de dato de float a int de la columna, para que coincida en ambos df
df2['id'] = df2['id'].astype(int)

In [29]:
#Realizo un merge entre los dos datasets que tenemos
df_final= pd.merge(df2, df, on='id')

In [30]:
#Se renombra las columnas necesarias
df_final.rename(columns={'crew':'director'}, inplace=True)
df_final.rename(columns={'cast':'actors'}, inplace=True)
df_final.rename(columns={'release_date':'release_year'}, inplace=True)

In [31]:
#Borre la columna belongs_to_collection
df_final.drop('belongs_to_collection', axis=1, inplace=True)

In [34]:
#Guardado de precaución
df_final.to_csv('Csv_Proyecto_Terminado2.csv', index=False)

In [36]:
df_final.head(3)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_year,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,actors,director
0,30000000.0,Animation,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0,12.451801,Tom Hanks,John Lasseter
1,65000000.0,Adventure,8844,en,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249.0,104.0,English,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,Robin Williams,Larry J. Franco
2,0.0,Romance,15602,en,A family wedding reignites the ancient feud be...,11.7129,Warner Bros.,United States of America,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,NaN,Walter Matthau,Howard Deutch
